## Import libs

In [1]:
import tensorflow as tf
#sess = tf.InteractiveSession()
#sess.close()

In [2]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.


In [3]:
import os
import urllib
import numpy as np
import math

In [4]:
from pyspark.sql import *
from pyspark import SparkConf,SparkContext

from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType ,DoubleType ,BooleanType, IntegerType
from pyspark.sql.functions import udf ,col ,upper, avg

from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

from pyspark.ml.classification import GBTClassifier
from pyspark.ml.regression import GBTRegressor ,LinearRegression ,GeneralizedLinearRegression
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.linalg import Vectors 
from pyspark.ml.linalg import VectorUDT

from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from pyspark.ml.clustering import KMeans

from pyspark.ml import Pipeline
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.regression import GBTRegressor ,LinearRegression ,GeneralizedLinearRegression


from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import RegressionMetrics

from pyspark.ml.feature import PCA

from pyspark.ml.linalg import Vectors 
from pyspark.ml.linalg import VectorUDT
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.feature import VectorAssembler

In [5]:
sc=SparkContext()
sqlContext=SQLContext(sc)

In [6]:
#For gpu accelerated NN training:
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.layers import Merge
from keras.regularizers import l2
from keras.optimizers import SGD

In [7]:
from sklearn.metrics import log_loss

In [9]:
print log_loss([0]*500  + [1]*500,[[.5,.5]]*1000)

print ""
print log_loss([0]*520  + [1]*480,[[.51,.49]]*1000)
print log_loss([0]*520  + [1]*480,[[.52,.48]]*1000)
print log_loss([0]*520  + [1]*480,[[.53,.47]]*1000)

print ""
print log_loss([0]*540  + [1]*460,[[.53,.47]]*1000)
print log_loss([0]*540  + [1]*460,[[.54,.46]]*1000)
print log_loss([0]*540  + [1]*460,[[.55,.45]]*1000)

print ""
print log_loss([0]*540  + [1]*460,[[.8,.2]]*540+([[.2,8]]*460))

0.69314718056

0.692547113878
0.69234696709
0.69254754212

0.690144655883
0.689943758458
0.690145520668

0.204365433835
9.99200722163e-16


In [11]:
#np.version

# Check that nvida drivers are working:

In [16]:
!nvidia-smi

Wed May 10 15:29:37 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 375.39                 Driver Version: 375.39                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 860M    Off  | 0000:01:00.0     Off |                  N/A |
| N/A   47C    P0    N/A /  N/A |    627MiB /  4044MiB |      6%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## Read input

In [17]:
#numerai_train_df     = sqlContext.read.option("header","true").csv('file:///home/garth/Downloads/numerai20170130/numerai_training_data.csv',inferSchema='true')
#numerai_tournament_df= sqlContext.read.option("header","true").csv('file:///home/garth/Downloads/numerai20170130/numerai_tournament_data.csv',inferSchema='true')

numerai_train_df     = sqlContext.read.option("header","true").csv('file:///home/garth/Downloads/numerai_data_2017-05-10/numerai_training_data.csv',inferSchema='true')
numerai_tournament_df= sqlContext.read.option("header","true").csv('file:///home/garth/Downloads/numerai_data_2017-05-10/numerai_tournament_data.csv',inferSchema='true')
                                                                   
print "length of training set:"
print (numerai_train_df.count())
print "length of tournament set:"
print (numerai_tournament_df.count())

length of training set:
108405
length of tournament set:
45822


In [19]:
numerai_train_df.show(3,False)
print ("____________________________________")
numerai_tournament_df.show(3,False)


+------+----+---------+--------+--------+--------+--------+--------+--------+--------+--------+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+------+
|id    |era |data_type|feature1|feature2|feature3|feature4|feature5|feature6|feature7|feature8|feature9|feature10|feature11|feature12|feature13|feature14|feature15|feature16|feature17|feature18|feature19|feature20|feature21|target|
+------+----+---------+--------+--------+--------+--------+--------+--------+--------+--------+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+------+
|140721|era1|train    |0.26647 |0.42487 |0.81401 |0.22889 |0.27456 |0.55654 |0.5531  |0.71899 |0.20035 |0.62848  |0.18636  |0.26651  |0.18847  |0.77235  |0.55002  |0.20237  |0.79605  |0.82971  |0.45757  |0.69761  |0.53739  |1     |
|6942  |era1|train    |0.41334 |0.47533 |0.71847 |0.40792 |0.32433 |0.55

## Convert input for use in ml lib algos

In [23]:
train_df=numerai_train_df.rdd.map(lambda x: (x.id,x.era,x.data_type,Vectors.dense([
                x.feature1
                ,x.feature2
                ,x.feature3
                ,x.feature4
                ,x.feature5
                ,x.feature6
                ,x.feature7
                ,x.feature8
                ,x.feature9
                ,x.feature10
                ,x.feature11
                ,x.feature12
                ,x.feature13
                ,x.feature14
                ,x.feature15
                ,x.feature16
                ,x.feature17
                ,x.feature18
                ,x.feature19
                ,x.feature20
                ,x.feature21
                ]),float(x.target))).toDF(["id","era","type","features","label"])

tournament_df=numerai_tournament_df.rdd.map(lambda x: (x.id,x.era,x.data_type,Vectors.dense([
                x.feature1
                ,x.feature2
                ,x.feature3
                ,x.feature4
                ,x.feature5
                ,x.feature6
                ,x.feature7
                ,x.feature8
                ,x.feature9
                ,x.feature10
                ,x.feature11
                ,x.feature12
                ,x.feature13
                ,x.feature14
                ,x.feature15
                ,x.feature16
                ,x.feature17
                ,x.feature18
                ,x.feature19
                ,x.feature20
                ,x.feature21
                ]))).toDF(["id","era","type","features"])



In [25]:
train_df.show()
tournament_df.show()

+------+----+-----+--------------------+-----+
|    id| era| type|            features|label|
+------+----+-----+--------------------+-----+
|140721|era1|train|[0.26647,0.42487,...|  1.0|
|  6942|era1|train|[0.41334,0.47533,...|  1.0|
| 97514|era1|train|[0.48937,0.5603,0...|  1.0|
| 60256|era1|train|[0.61195,0.65958,...|  1.0|
| 30143|era1|train|[0.43758,0.50085,...|  1.0|
| 91955|era1|train|[0.46868,0.52025,...|  0.0|
| 73822|era1|train|[0.47625,0.74148,...|  0.0|
|141352|era1|train|[0.49953,0.5555,0...|  0.0|
|  2202|era1|train|[0.35671,0.37318,...|  0.0|
|  1860|era1|train|[0.52146,0.51701,...|  1.0|
|117896|era1|train|[0.43264,0.53669,...|  1.0|
|128292|era1|train|[0.52781,0.48414,...|  1.0|
| 91101|era1|train|[0.80581,0.59023,...|  1.0|
|110379|era1|train|[0.55122,0.60682,...|  1.0|
|125670|era1|train|[0.60113,0.27452,...|  1.0|
| 24197|era1|train|[0.50609,0.50197,...|  0.0|
| 90893|era1|train|[0.39845,0.46492,...|  1.0|
| 47735|era1|train|[0.48893,0.54007,...|  0.0|
|109070|era1|

In [26]:
#(trainingData, testData) = train_df.randomSplit([0.6, 0.4],123456)
(trainingData, testData,everythingElse) = train_df.randomSplit([0.2, 0.1, 0.7],12345)

## Define log loss function

In [27]:
import scipy as sp
def logloss(act, pred):
    epsilon = 1e-15
    pred = sp.maximum(epsilon, pred)
    pred = sp.minimum(1-epsilon, pred)
    ll = sum(act*sp.log(pred) + sp.subtract(1,act)*sp.log(sp.subtract(1,pred)))
    ll = ll * -1.0/len(act)
    return ll

## PCA

In [ ]:
# Do PCA and rename REPLACE the features column

pca = PCA(k=10, inputCol="features", outputCol="pcaFeatures")
model = pca.fit(trainingData)
trainingDataPCA = model.transform(trainingData).drop("features")
trainingDataPCA = trainingDataPCA.withColumn("features",trainingDataPCA.pcaFeatures).drop("pcaFeatures")

testingDataPCA = model.transform(testData).drop("features")
testingDataPCA = testingDataPCA.withColumn("features",testingDataPCA.pcaFeatures).drop("pcaFeatures")

tournamentDataPCA = model.transform(tournament_df).drop("features")
tournamentDataPCA = tournamentDataPCA.withColumn("features",tournamentDataPCA.pcaFeatures).drop("pcaFeatures")

In [ ]:
#testingDataPCA.show(1,False)

In [ ]:
trainingData = trainingDataPCA
testData = testingDataPCA
tournament_df =tournamentDataPCA

## Clustering

In [ ]:
k=10
#k=5
#k=50
threshold1=0.69
threshold2=0.692

In [ ]:
kmeans = KMeans().setK(k).setSeed(1)
model = kmeans.fit(trainingData)
wssse = model.computeCost(trainingData)
print("Within Set Sum of Squared Errors = " + str(wssse))
centers = model.clusterCenters()

## Optimise for the number of clusters by finding the inflection point in cost vs number of clusters

In [ ]:
clustered_train_df = model.transform(trainingData)
clustered_train_df = clustered_train_df.withColumn("clusterNum",clustered_train_df.prediction).drop("prediction")
clustered_train_df.show(1)

clustered_test_df = model.transform(testData)
clustered_test_df = clustered_test_df.withColumn("clusterNum",clustered_test_df.prediction).drop("prediction")
clustered_test_df.show(1)

clustered_tournament_df = model.transform(tournament_df)
clustered_tournament_df = clustered_tournament_df.withColumn("clusterNum",clustered_tournament_df.prediction).drop("prediction")
clustered_tournament_df.show(1)


clustered_test_df.describe().show()

# Neural Net with Keras

In [ ]:
seed = 7
np.random.seed(seed)

In [ ]:
assembler = VectorAssembler(
    inputCols=["features", "clusterNum"],
    outputCol="features2")
clustered_train_df2 = assembler.transform(clustered_train_df)
clustered_test_df2  = assembler.transform(clustered_test_df)

In [ ]:
clustered_train_df2.show(1)
clustered_test_df2.show(1)

In [ ]:
features    = np.array(clustered_train_df2.select("features2").toPandas())
labels      = np.array(clustered_train_df2.select("label").toPandas())

featuresT    = np.array(clustered_test_df2.select("features2").toPandas())
labelsT      = np.array(clustered_test_df2.select("label").toPandas())

In [ ]:
train_np     = np.array(map(lambda x: np.array(map(lambda y: np.array(y),x)), features))[:,][:][:]
#train_np_pca = np.array(map(lambda x: np.array(map(lambda y: np.array(y),x)), pcaFeatures))[:,][:][:]
train_res= np.array(map(lambda x: np.array(map(lambda y: np.array(y),x)), labels))[:,0]

In [ ]:
train_np_test     = np.array(map(lambda x: np.array(map(lambda y: np.array(y),x)), featuresT))[:,0][:][:]
#train_np_pca_test = np.array(map(lambda x: np.array(map(lambda y: np.array(y),x)), pcaFeaturesT))[:,0][:][:]
train_res_test    = np.array(map(lambda x: np.array(map(lambda y: np.array(y),x)), labelsT))[:,0]

In [ ]:
print type(train_np[0])
print train_np[10]

print train_res[10]
sum1= float(sum(train_res))
len1= float(len(train_res))
print "ratio: "+str(sum1/len1)

In [ ]:
print train_np.shape
train_np=train_np.reshape(train_np.shape[0],train_np.shape[2])
print train_np.shape

#print train_np_pca.shape
#train_np_pca=train_np_pca.reshape(train_np_pca.shape[0],train_np_pca.shape[2])
#print train_np_pca.shape

In [ ]:
def create_model():
    model = Sequential()
    #model.add(Dense(11, input_dim=11, init='uniform', activation='relu'))
    model.add(Dense(11, input_dim=8, init='uniform', activation='relu'))
    model.add(Dense(11, init='uniform', activation='relu'))
    model.add(Dense(1, init='uniform', activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
dataset = np.loadtxt("/home/garth/Downloads/pima-indians-diabetes.data.txt", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]

In [ ]:
#model.fit(train_np, train_res, nb_epoch=60, batch_size=1200)

evaluated_train = model.evaluate(train_np,train_res)
evaluated_test  = model.evaluate(train_np_test,train_res_test)

nn_answer       = model.predict(train_np_test)

print ""
print "---------------Train-------------------"
print "logloss : "+str(evaluated_train[0])
print "accuracy: "+str(evaluated_train[1]*100)+"%"
print "---------------Test--------------------"
print "logloss : "+str(evaluated_test[0])
print "accuracy: "+str(evaluated_test[1]*100)+"%"

In [ ]:
print nn_answer

# XGB

In [ ]:
import xgboost
from sklearn.metrics import accuracy_score

In [ ]:
model = xgboost.XGBClassifier()

#model.fit(train_np_pca, train_res)
model.fit(train_np, train_res)

# make predictions for test data
#y_pred = model.predict(train_np_pca_test)
y_pred = model.predict(train_np_test)

xgb_predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(train_res_test, xgb_predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
print type(np.array(xgb_predictions))
print type(nn_answer)

nn_answer=nn_answer.reshape(nn_answer.shape[0],)

print np.array(xgb_predictions)
print nn_answer

In [ ]:
trainingData.show()

train_xgb_udf = udf(lambda a: model.predict(a) , DoubleType())

trainingData2=trainingData.withColumn("xgboost_pred",train_xgb_udf(trainingData.features))

In [ ]:
train_keras_udf = udf(lambda a,b: final_model. , DoubleType())

In [ ]:
trainingData3=trainingData.withColumn("keras_pred",train_keras_udf(trainingData.features,trainingData.pcaFeatures))

# Bucket

In [ ]:
evaluated  = final_model.predict([train_np_test,train_np_pca_test])

In [ ]:
print evaluated.shape
print train_res_test.shape

In [ ]:
evaluated=evaluated.reshape(evaluated.shape[0],)

In [ ]:
evaluated

In [ ]:
train_res_test

In [ ]:
ensemble = evaluated

In [ ]:
#trainingData2.show()

In [1]:
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
# Function to create model, required for KerasClassifier

Using TensorFlow backend.


In [2]:
from keras.wrappers.scikit_learn import BaseWrapper
import copy

def custom_get_params(self, **params):
    res = copy.deepcopy(self.sk_params)
    res.update({'build_fn': self.build_fn})
    return res

BaseWrapper.get_params = custom_get_params

In [3]:

def create_model():
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load dataset
dataset = numpy.loadtxt("/home/garth/Downloads/pima-indians-diabetes.data.txt", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]
# create model
model = KerasClassifier(build_fn=create_model, verbose=0)
# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, nb_epoch=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

JoblibInternalError: JoblibInternalError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/usr/lib/python2.7/runpy.py in _run_module_as_main(mod_name='ipykernel.__main__', alter_argv=1)
    169     pkg_name = mod_name.rpartition('.')[0]
    170     main_globals = sys.modules["__main__"].__dict__
    171     if alter_argv:
    172         sys.argv[0] = fname
    173     return _run_code(code, main_globals, None,
--> 174                      "__main__", fname, loader, pkg_name)
        fname = '/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py'
        loader = <pkgutil.ImpLoader instance>
        pkg_name = 'ipykernel'
    175 
    176 def run_module(mod_name, init_globals=None,
    177                run_name=None, alter_sys=False):
    178     """Execute a module's code without importing it

.......................................................

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
import numpy as np

seed = 7
numpy.random.seed(seed)


In [ ]:
dataset = np.loadtxt("/home/garth/Downloads/pima-indians-diabetes.data.txt", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]

In [ ]:
model = Sequential()
model.add(Dense(12, input_dim=8, init='uniform', activation='relu'))
model.add(Dense(8, init='uniform', activation='relu'))
model.add(Dense(1, init='uniform', activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X, Y, nb_epoch=150, batch_size=10)

In [ ]:
scores = model.evaluate(X, Y)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [7]:
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
# Function to create model, required for KerasClassifier
def create_model(optimizer='adam'):
    # create model
	model = Sequential()
	model.add(Dense(12, input_dim=8, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
	return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load dataset
dataset = numpy.loadtxt("/home/garth/Downloads/pima-indians-diabetes.data.txt", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]
# create model


model = KerasClassifier(build_fn=create_model, nb_epoch=100, batch_size=10, verbose=0)
# define the grid search parameters
optimizer = ['SGD', 'RMSprop', 'Adagrad']
param_grid = dict(optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

JoblibInternalError: JoblibInternalError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/usr/lib/python2.7/runpy.py in _run_module_as_main(mod_name='ipykernel.__main__', alter_argv=1)
    169     pkg_name = mod_name.rpartition('.')[0]
    170     main_globals = sys.modules["__main__"].__dict__
    171     if alter_argv:
    172         sys.argv[0] = fname
    173     return _run_code(code, main_globals, None,
--> 174                      "__main__", fname, loader, pkg_name)
        fname = '/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py'
        loader = <pkgutil.ImpLoader instance>
        pkg_name = 'ipykernel'
    175 
    176 def run_module(mod_name, init_globals=None,
    177                run_name=None, alter_sys=False):
    178     """Execute a module's code without importing it

.......................................................